# MTA Turnstile Project

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime as dt
import datetime 

%matplotlib inline

In [ ]:
# Source: http://web.mta.info/developers/turnstile.html
def get_data(week_nums):
    url = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt"
    dfs = []
    for week_num in week_nums:
        file_url = url.format(week_num)
        dfs.append(pd.read_csv(file_url))
    return pd.concat(dfs)
        
week_nums = [180505, 180512, 180519, 180526, 180602]
turnstiles_df = get_data(week_nums)

In [ ]:
# Aside on string formatting in Python 

my_string = "Hello, {}. Are you {} to be learning {}?"
print(my_string.format("Lara", "delighted", "Python"))

In [ ]:
turnstiles_df.head()

In [ ]:
# Check whether there's any formatting issues in the column names 

turnstiles_df.columns

In [ ]:
turnstiles_df.columns = [column.strip() for column in turnstiles_df.columns]

In [ ]:
turnstiles_df.columns

In [ ]:
turnstiles_df.head()

In [ ]:
turnstiles_df.tail()

In [ ]:
# Three weeks of data
turnstiles_df.DATE.value_counts().sort_index()

In [ ]:
turnstiles_df.columns

In [ ]:
mask = ((turnstiles_df["C/A"] == "A002") & 
        (turnstiles_df["UNIT"] == "R051") & 
        (turnstiles_df["SCP"] == "02-00-00") & 
        (turnstiles_df["STATION"] == "59 ST"))

turnstiles_df[mask].head()

In [ ]:
# Take the date and time fields into a single datetime column
turnstiles_df["DATE_TIME"] = pd.to_datetime(turnstiles_df.DATE + " " + turnstiles_df.TIME, format="%m/%d/%Y %H:%M:%S")

In [ ]:
mask = ((turnstiles_df["C/A"] == "R626") & 
(turnstiles_df["UNIT"] == "R062") & 
(turnstiles_df["SCP"] == "00-00-00") & 
(turnstiles_df["STATION"] == "CROWN HTS-UTICA"))
turnstiles_df[mask].head()

In [ ]:
# Sanity check to verify that "C/A", "UNIT", "SCP", "STATION", "DATE_TIME" is unique
(turnstiles_df
 .groupby(["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"])  
 .ENTRIES.count()
 .reset_index() # or use as_index = False; otherwise makes groupby columns new index 
 .sort_values("ENTRIES", ascending=False)).head(5)

In [ ]:
# On 9/16, we seem to have two entries for same time.  Let's take a look
mask = ((turnstiles_df["C/A"] == "R504") & 
(turnstiles_df["UNIT"] == "R276") & 
(turnstiles_df["SCP"] == "00-00-01") & 
(turnstiles_df["STATION"] == "VERNON-JACKSON") &
(turnstiles_df["DATE_TIME"].dt.date == datetime.datetime(2016, 9, 16).date()))
turnstiles_df[mask].head()

* Looks to be a incorrect AUD entry.  May be we should just select the Regular one.

In [ ]:
turnstiles_df.DESC.value_counts()

* Since we are only interested in Entries, we might be OK.

In [ ]:
# Get rid of the duplicate entry
turnstiles_df.sort_values(["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"], inplace=True, \
                          ascending=False)
turnstiles_df.drop_duplicates(subset=["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"], inplace=True)

In [ ]:
# Sanity check to verify that "C/A", "UNIT", "SCP", "STATION", "DATE_TIME" is unique
(turnstiles_df
 .groupby(["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"])
 .ENTRIES.count()
 .reset_index()
 .sort_values("ENTRIES", ascending=False)).head(5)

* No more duplicate Entries

In [ ]:
# Drop Exits and Desc columns.  To prevent errors in multiple run of cell, 
# errors on drop is ignored (e.g. if some columns were dropped already)
turnstiles_df = turnstiles_df.drop(["EXITS", "DESC"], axis=1, errors="ignore")

In [ ]:
turnstiles_df.dtypes

In [ ]:
turnstiles_daily = turnstiles_df.groupby(["C/A", "UNIT", "SCP", "STATION", "DATE"])\
.ENTRIES.first().reset_index()

In [ ]:
turnstiles_daily.head()

In [ ]:
# .groupby returns a group object whose values are the index values belonging to that group

#turnstiles_daily.groupby(["C/A", "UNIT", "SCP", "STATION"]).groups

In [ ]:
turnstiles_daily[["PREV_DATE", "PREV_ENTRIES"]] = (turnstiles_daily
                                                       .groupby(["C/A", "UNIT", "SCP", "STATION"])["DATE", "ENTRIES"]
                                                       .transform(lambda grp: grp.shift(1)))
# transform() takes a function as parameter
# shift moves the index by the number of periods given (positive or negative)

In [ ]:
turnstiles_daily.head()

In [ ]:
turnstiles_daily.tail()

In [ ]:
# Drop the rows for first date
turnstiles_daily.dropna(subset=["PREV_DATE"], axis=0, inplace=True)
# axis = 0 means index (=1 means column)

In [ ]:
# let's check that the number of entries for today is higher than entries for yesterday
turnstiles_daily[turnstiles_daily["ENTRIES"] < turnstiles_daily["PREV_ENTRIES"]].head()

In [ ]:
# What's the deal with counter being in reverse
mask = ((turnstiles_df["C/A"] == "A011") & 
(turnstiles_df["UNIT"] == "R080") & 
(turnstiles_df["SCP"] == "01-00-00") & 
(turnstiles_df["STATION"] == "57 ST-7 AV") &
(turnstiles_df["DATE_TIME"].dt.date == datetime.datetime(2016, 8, 27).date())) 
# datetime is both name of module and name of constructor of datetime object
turnstiles_df[mask].head()

* Counter working in Reverse??? - WHHHAT

In [ ]:
# Let's see how many stations have this problem

(turnstiles_daily[turnstiles_daily["ENTRIES"] < turnstiles_daily["PREV_ENTRIES"]]
    .groupby(["C/A", "UNIT", "SCP", "STATION"])
     .size()) # size() behaves same as if we'd done .DATE.count() 

In [ ]:
def get_daily_counts(row, max_counter):
    counter = row["ENTRIES"] - row["PREV_ENTRIES"]
    if counter < 0:
        counter = -counter
    if counter > max_counter:
        print(row["ENTRIES"], row["PREV_ENTRIES"])
        return 0
    return counter

# If counter is > 1 million, then the counter might have been reset.  
# Just set it to zero as different counters have different cycle limits
_ = turnstiles_daily.apply(get_daily_counts, axis=1, max_counter=1000000) 

# apply takes a function as parameter and applies it along the given axis (1=apply by row) 
# apply takes care of passing each row to the function

In [ ]:
def get_daily_counts(row, max_counter):
    counter = row["ENTRIES"] - row["PREV_ENTRIES"]
    if counter < 0:
        # Maybe counter is reversed?
        counter = -counter
    if counter > max_counter:
        print(row["ENTRIES"], row["PREV_ENTRIES"])
        counter = min(row["ENTRIES"], row["PREV_ENTRIES"])
        # if current entries is bad, use yesterday's count as proxy
    if counter > max_counter:
        # Check it again to make sure we are not giving a counter that's too big
        return 0
    return counter

# If counter is > 1Million, then the counter might have been reset.  
# Just set it to zero as different counters have different cycle limits
turnstiles_daily["DAILY_ENTRIES"] = turnstiles_daily.apply(get_daily_counts, axis=1, max_counter=1000000)

In [ ]:
turnstiles_daily.head()

In [ ]:
nyc_turnstile_geocode_df = pd.read_csv('https://github.com/chriswhong/nycturnstiles/raw/master/geocoded.csv', \
                                       names = ['UNIT', 'C/A', 'STATION', 'LINENAME', 'DIVISION', 'LAT', 'LONG'], 
                                       index_col=False)

In [ ]:
nyc_turnstile_geocode_df.reindex()

In [ ]:
merged = turnstiles_df.merge(nyc_turnstile_geocode_df, 
                                on=['STATION', 'UNIT', 'LINENAME','C/A', 'DIVISION'], 
                                how='left')
merged.head(5)

In [ ]:
mask = ((turnstiles_daily["C/A"] == "A002") & 
(turnstiles_daily["UNIT"] == "R051") &  
(turnstiles_daily["STATION"] == "59 ST") 
)
turnstiles_daily[mask].head()

In [ ]:
mask2 = (#(nyc_turnstile_geocode_df["C/A"] == "A002") & 
(nyc_turnstile_geocode_df["UNIT"] == "R051") &  
(nyc_turnstile_geocode_df["STATION"] == "59 ST") 
)
nyc_turnstile_geocode_df[mask2].head()

In [ ]:
#merged.to_csv('merged.csv')

In [ ]:
#!scrapy startproject nyc_tech_companies


# import scrapy


# class CompaniesSpider(scrapy.Spider):
#     name = "companies"
#     start_urls = ['https://www.builtinnyc.com/2017/11/07/nyc-top-100-tech-companies-2017',
#     ]

#     def parse(self, response):
#         companies = response.css('.info .title a::text').extract()
#         lines = response.xpath('//strong/parent::p[text()]').extract()
#         splitted = [line.split() for line in lines]
#         splitted = [each[4] for each in splitted]
#         splitted = [number.replace(',', '') for number in splitted]
#         splitted = [number.replace('*', '') for number in splitted]
#         number_of_employees = [int(num) for num in splitted]
#         result = dict(zip(companies, number_of_employees))
#         urls = response.css('.info .link a::attr(href)').extract()
#         def address_parse(response):
#             address = response.css('.gmap_location_widget_address::text').extract_first()
#             company = response.css('h1::text').extract_first()
#             return {'company': company,
#                 'address': address,
#                 'number_of_employees': result[company]}
#         for url in urls:
#             yield scrapy.Request(url, callback=address_parse)


#!scrapy crawl nyc_tech_companies

null = None
companies_dict = [
{"company": "OnDeck", "address": "New York, NY", "number_of_employees": 250},
{"company": "Warby Parker", "address": "161 Avenue of the Americas, New York City, NY 10013", "number_of_employees": 251},
{"company": "COMPLEX (dba Complex Media, Inc.)", "address": null, "number_of_employees": 250},
{"company": "Bloomberg", "address": "731 Lexington Avenue, New York, NY 10022", "number_of_employees": 9000},
{"company": "BounceX", "address": "620 8th Ave, New York, NY 10018", "number_of_employees": 250},
{"company": "Handy", "address": "New York , NY", "number_of_employees": 255},
{"company": "Vimeo", "address": null, "number_of_employees": 240},
{"company": "1010data", "address": "750 3rd Ave., 4th Floor, New York, NY 10017", "number_of_employees": 260},
{"company": "Justworks", "address": "601 W 26th St, New York, NY 10001", "number_of_employees": 288},
{"company": "Fareportal", "address": "135 W 50 Street #500, New York, NY 10020", "number_of_employees": 300},
{"company": "1stdibs", "address": "51 Astor Place, New York, NY 10003", "number_of_employees": 269},
{"company": "Integral Ad Science", "address": "95 Morton Street, New York , NY 10014", "number_of_employees": 262},
{"company": "MongoDB", "address": "New York , NY", "number_of_employees": 300},
{"company": "Sisense", "address": " 1359 Broadway 4th Floor , New York, NY 10018", "number_of_employees": 122},
{"company": "Compass", "address": "90 Fifth Avenue, 3rd Floor, New York, NY 10011", "number_of_employees": 300},
{"company": "Casper", "address": "New York, NY", "number_of_employees": 300},
{"company": "Mediaocean", "address": "New York, NY", "number_of_employees": 330},
{"company": "Tapad", "address": "New York , NY", "number_of_employees": 125},
{"company": "Yodle", "address": "330 W 34th Street, New York, NY 10001", "number_of_employees": 124},
{"company": "Sailthru", "address": null, "number_of_employees": 130},
{"company": "Schoology", "address": "2 Penn Plaza, 10th Floor, New York, NY 10121", "number_of_employees": 131},
{"company": "Newsela", "address": "475 Tenth Avenue, New York, NY 10018", "number_of_employees": 135},
{"company": "Spring", "address": "New York , NY", "number_of_employees": 131},
{"company": "Birchbox", "address": "28 E 28th St, New York, NY 10016", "number_of_employees": 130},
{"company": "MediaRadar", "address": "New York, NY", "number_of_employees": 133},
{"company": "Percolate", "address": "New York , NY", "number_of_employees": 140},
{"company": "CB Insights", "address": "498 7th Avenue, 17th Floor, New York, NY 10018", "number_of_employees": 136},
{"company": "Annalect", "address": "195 Broadway, New York, NY 10007", "number_of_employees": 135},
{"company": "Stack Overflow", "address": "110 William Street , New York, NY 10038", "number_of_employees": 140},
{"company": "Purch", "address": null, "number_of_employees": 132},
{"company": "Transfix", "address": "498 7th Avenue, New York City, NY 10018", "number_of_employees": 140},
{"company": "SeatGeek", "address": "New York , NY", "number_of_employees": 138},
{"company": "Bluecore", "address": "116 Nassau Street, New York , NY 10038", "number_of_employees": 145},
{"company": "Quartet", "address": "119 W 40th St, New York , NY 10036", "number_of_employees": 145},
{"company": "ShopKeep", "address": "143 Varick Street, New York, NY 10013", "number_of_employees": 140},
{"company": "FanDuel", "address": "1375 Broadway, New York , NY 10018", "number_of_employees": 143},
{"company": "Plated", "address": "22 West 19th Street, 5th Floor, New York , NY 10011", "number_of_employees": 140},
{"company": "Managed by Q", "address": "233 Spring St., New York, NY 10012", "number_of_employees": 160},
{"company": "LiveIntent", "address": null, "number_of_employees": 149},
{"company": "General Assembly", "address": "New York, NY", "number_of_employees": 146},
{"company": "Quartz", "address": null, "number_of_employees": 150},
{"company": "Artnet", "address": "233 Broadway 26th Floor, New York, NY 10279", "number_of_employees": 150},
{"company": "Artsy", "address": "401 Broadway, 25th Floor, New York, NY 10013", "number_of_employees": 146},
{"company": "Glossier", "address": " 123 Lafayette St., Penthouse, New York, NY 10013", "number_of_employees": 163},
{"company": "Dataminr", "address": "6 East 32nd St, New York , NY 10016", "number_of_employees": 170},
{"company": "VTS", "address": "114 West 41st Street, New York City, NY 10018", "number_of_employees": 172},
{"company": "ConsenSys", "address": "Brooklyn, NY", "number_of_employees": 164},
{"company": "Outbrain", "address": "39 West 13th Street, New York, NY", "number_of_employees": 160},
{"company": "Conductor", "address": "2 Park Ave 15th Floor , New York , NY", "number_of_employees": 166},
{"company": "Foursquare", "address": "568 Broadway, New York , NY 10012", "number_of_employees": 177},
{"company": "Braze", "address": " 318 W 39th St, 5th Floor, New York, NY 10018", "number_of_employees": 175},
{"company": "emarketer", "address": null, "number_of_employees": 167},
{"company": "Zeta Interactive", "address": "New York , NY", "number_of_employees": 175},
{"company": "Meetup", "address": "632 Broadway, New York, NY 10012", "number_of_employees": 186},
{"company": "Makerbot", "address": "1 MetroTech Center, Brooklyn, NY 11201", "number_of_employees": 194},
{"company": "JW Player", "address": "2 Park Ave, New York, NY 10016", "number_of_employees": 185},
{"company": "Kargo", "address": "826 Broadway, New York, NY", "number_of_employees": 182},
{"company": "Greenhouse Software", "address": "110 5th Avenue, 3rd Floor, New York , NY 10011", "number_of_employees": 175},
{"company": "Harry's", "address": "75 Varick Street, New York , NY 10013", "number_of_employees": 197},
{"company": "Undertone", "address": "New York, NY", "number_of_employees": 185},
{"company": "Mashable", "address": "East 23rd Street, New York , NY", "number_of_employees": 184},
{"company": "Moat", "address": "New York, NY", "number_of_employees": 187},
{"company": "DigitalOcean", "address": "101 Avenue of the Americas, New York , NY 10013", "number_of_employees": 200},
{"company": "Uncommon Goods", "address": null, "number_of_employees": 196},
{"company": "ClassPass", "address": "New York , NY", "number_of_employees": 200},
{"company": "Taboola", "address": "19 W. 22nd Street 5th Floor, New York , NY", "number_of_employees": 198},
{"company": "Rent the Runway", "address": "345 Hudson Street , New York , NY 10014", "number_of_employees": 230},
{"company": "TMP Worldwide", "address": "125 Broad St. 10th Floor, New York, NY 10004", "number_of_employees": 201},
{"company": "2U", "address": "New York, NY", "number_of_employees": 200},
{"company": "LearnVest", "address": "41 East 11th Street, 2nd Floor, New York, NY 10003", "number_of_employees": 236},
{"company": "Betterment", "address": "New York, NY", "number_of_employees": 216},
{"company": "Bonobos", "address": "45 W 25th Street, 5th Floor, New York, NY 10010", "number_of_employees": 225},
{"company": "Sprinklr", "address": "New York , NY", "number_of_employees": 218},
{"company": "Datadog", "address": "New York, NY", "number_of_employees": 225},
{"company": "Presidio", "address": null, "number_of_employees": 332},
{"company": "Namely", "address": "195 Broadway, New York, NY 10007", "number_of_employees": 380},
{"company": "MediaMath", "address": "New York, NY", "number_of_employees": 346},
{"company": "Zocdoc", "address": "568 Broadway, New York , NY 10012", "number_of_employees": 340},
{"company": "XO Group Inc", "address": "195 Broadway, New York, NY 10007", "number_of_employees": 350},
{"company": "Group Nine Media", "address": "568 Broadway Ave, New York, NY 10012", "number_of_employees": 420},
{"company": "Intersection", "address": "10 Hudson Yards, New York City, NY 10001", "number_of_employees": 380},
{"company": "Gilt Groupe", "address": "250 Vesey Street, 21st Floor, New York, NY 10281", "number_of_employees": 400},
{"company": "Squarespace", "address": "8 Clarkson Street, New York, NY 10014", "number_of_employees": 426},
{"company": "The Business Insider", "address": "150 Fifth Avenue 8th floor, New York , NY", "number_of_employees": 350},
{"company": "Flatiron Health", "address": "One Soho Square at 233 Spring Stree, New York, NY 10013", "number_of_employees": 460},
{"company": "Oscar", "address": "295 Lafayette St, New York, NY 10012", "number_of_employees": 400},
{"company": "Infor", "address": null, "number_of_employees": 420},
{"company": "Shutterstock", "address": "350 5th Avenue, New York, NY 10118", "number_of_employees": 450},
{"company": "Information Builders", "address": null, "number_of_employees": 485},
{"company": "Peloton", "address": "125 w 25th St, New York , NY", "number_of_employees": 515},
{"company": "Yext", "address": "1 Madison Avenue, New York, NY 10010", "number_of_employees": 675},
{"company": "Refinery29", "address": "225 Broadway Floor 23 , New York , NY", "number_of_employees": 558},
{"company": "FreshDirect", "address": null, "number_of_employees": 657},
{"company": "AppNexus", "address": "New York , NY", "number_of_employees": 583},
{"company": "Etsy", "address": "117 Adams Street, Brooklyn, NY 11201", "number_of_employees": 622},
{"company": "Blue Apron", "address": "New York, NY", "number_of_employees": 890},
{"company": "Vice Media", "address": "49 S 2nd St, Brooklyn, NY 11249", "number_of_employees": 1217},
{"company": "E*Trade", "address": null, "number_of_employees": 827},
{"company": "BuzzFeed", "address": "111 E 18th St, New York, NY 10003", "number_of_employees": 730},
{"company": "CA Technologies", "address": null, "number_of_employees": 1230},
{"company": "Oath", "address": "770 Broadway, New York, NY 10003-9562", "number_of_employees": 1400}
]



In [ ]:
company_data = pd.DataFrame(companies_dict)
company_data.head()

In [ ]:
company_data.address.value_counts(dropna=False)

In [ ]:
company_data.loc[company_data.company=='CA Technologies', 'address'] = '520 Madison Ave, New York, NY 10022'
company_data.loc[company_data.company=='Blue Apron', 'address'] = '40 W 23rd St 5th floor, New York, NY'
company_data.loc[company_data.company=='E*Trade', 'address'] = '11 Times Sq, New York, NY 10036'
company_data.loc[company_data.company=='AppNexus', 'address'] = '28 W 23rd St, New York, NY 10010'
company_data.loc[company_data.company=='Information Builders', 'address'] = '2 Pennsylvania Plaza, New York, NY 10121'
company_data.loc[company_data.company=='Infor', 'address'] = '641 6th Ave, New York, NY 10011'
company_data.loc[company_data.company=='FreshDirect', 'address'] = '2330 Borden Ave Long Island City, NY 11101'
company_data.loc[company_data.company=='Refinery29', 'address'] = '225 Broadway, New York , NY'
company_data.loc[company_data.company=='Flatiron Health', 'address'] = '233 Spring Street, New York, NY'
company_data.loc[company_data.company=='Blue Apron', 'address'] = '40 W 23rd St, New York, NY'

In [ ]:
companies_20 = company_data.sort_values('number_of_employees', ascending=False).head(20)

In [ ]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="NYC_Tech_companies")

In [ ]:
from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=2)

In [ ]:
companies_20['location'] = companies_20['address'].apply(geocode)
companies_20['point'] = companies_20['location'].apply(lambda loc: tuple(loc.point) if loc else None)

In [ ]:
companies_20.head()

In [ ]:
#companies_20.to_csv('companies_20.csv')

In [ ]:
from scipy.spatial.distance import cdist

def closest_point(point, points):
    """ Find closest point from a list of points. """
    return points[cdist([point], points).argmin()]

In [ ]:
nyc_turnstile_geocode_df['point'] = [
    (x,y) for x, y in zip(nyc_turnstile_geocode_df['LAT'], nyc_turnstile_geocode_df['LONG'])]

In [ ]:
nyc_turnstile_geocode_df.tail()

In [ ]:
# remove the last coordinate in the geocoded points
companies_20['point'] = [(x,y) for x,y,z in companies_20['point']]

In [ ]:
companies_20['closest'] = [closest_point(x, list(nyc_turnstile_geocode_df['point'][:-18])) \
                           for x in companies_20['point']]

In [ ]:
merged_companies = companies_20.merge(nyc_turnstile_geocode_df, how='left', left_on='closest', right_on='point')

In [ ]:
merged_companies.columns

In [ ]:
final_df = merged_companies[['address', 'company', 'number_of_employees', 'location', 'point_x','closest', 'STATION']]

In [ ]:
final_df.drop_duplicates(subset=['address', 'company', 'number_of_employees', \
                                 'point_x','closest', 'STATION'], \
                         inplace = True)

In [ ]:
final_df.drop(final_df.index[1], inplace=True)

In [ ]:
final_df.head()

In [ ]:
top_20_cmpy_df = final_df.reset_index(drop=True)

In [ ]:
top_20_cmpy_df.rename(columns={'point_x': 'company_coordinates', 'STATION':'station'}, inplace=True)

In [ ]:
top_20_cmpy_df.head()

In [ ]:
#top_20_cmpy_df.to_pickle('top_20_cmpy_df.pkl')

In [ ]:
plt.bar(list(top_20_cmpy_df['company']), list(top_20_cmpy_df['number_of_employees']), label='company')
plt.xticks(rotation=90)
#plt.savefig('temp.png',bbox_inches='tight', pad_inches=1)

In [ ]:
turnstiles_daily_stations = turnstiles_daily.sort_values('DAILY_ENTRIES', ascending=False).groupby('STATION')\
    .agg({'DAILY_ENTRIES': sum})\
    .sort_values('DAILY_ENTRIES', ascending=False)\
    .rename(columns={'DAILY_ENTRIES': 'MONTHLY_ENTRIES'})

In [ ]:
plt.bar(list(turnstiles_daily_stations.index)[:20],
        list(turnstiles_daily_stations.head(20)['MONTHLY_ENTRIES']), label='company')
plt.xticks(rotation=90)
#plt.savefig('temp_stations.png',bbox_inches='tight', pad_inches=1)

In [ ]:
import folium
df = top_20_cmpy_df
m = folium.Map(location=[40.730610, -73.935242])

for i, row in df.iterrows():
   folium.Marker(location = row['company_coordinates'],
           popup = str('<strong>Company:  </strong> ' + row['company'] + '<br>' + '<strong>Nearest Station:  </strong> ' + row['station'] \
                       + '<br>' + '<strong>Number of Employees:  </strong>' + str(row['number_of_employees'])),

                         icon= folium.Icon(icon = 'info-sign')).add_to(m)
   folium.Marker(location = row['closest'],
                popup = str('<strong>Station:  </strong> ' + row['station']),
                icon = folium.Icon(color='red', icon='info-sign')).add_to(m)
    
m